Copyright 2021-2022 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of Cybertron package.

The Cybertron is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

Cybertron tutorial 02: Setup for model and readout

In [1]:
import sys
import time
import numpy as np
from mindspore import nn
from mindspore import context
from mindspore import dataset as ds
from mindspore.train import Model
from mindspore.train.callback import LossMonitor
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig
from cybertron import Cybertron
from cybertron import MolCT
from cybertron import AtomwiseReadout
from cybertron.train import WithLabelLossCell

context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

[WARNING] ME(10115:139887459878720,MainProcess):2022-08-10-17:07:27.294.144 [mindspore/run_check/_check_version.py:137] Can not found cuda libs, please confirm that the correct cuda version has been installed, you can refer to the installation guidelines: https://www.mindspore.cn/install


In [2]:
train_file = sys.path[0] + '/dataset_qm9_origin_trainset_1024.npz'

train_data = np.load(train_file)

idx = [7]  # U0

num_atom = int(train_data['num_atoms'])
scale = train_data['scale'][idx]
shift = train_data['shift'][idx]
ref = train_data['type_ref'][:, idx]

In [3]:
mod = MolCT(
    cutoff=1,
    n_interaction=3,
    dim_feature=128,
    n_heads=8,
    fixed_cycles=False,
    activation='swish',
    max_cycles=10,
    length_unit='nm',
)

In [4]:
readout = AtomwiseReadout(
    mod, dim_output=1, scale=scale, shift=shift, type_ref=ref, energy_unit='kj/mol')

In [5]:
net = Cybertron(model=mod, readout=readout, dim_output=1,
                num_atoms=num_atom, length_unit='nm', energy_unit='kj/mol')
net.print_info()

Cybertron Engine, Ride-on!
--------------------------------------------------------------------------------
    Length unit: nm
    Input unit scale: 1
--------------------------------------------------------------------------------
    Deep molecular model:  MolCT
--------------------------------------------------------------------------------
       Length unit: nm
       Atom embedding size: 64
       Cutoff distance: 1.0 nm
       Radical basis function (RBF): LogGaussianBasis
          Minimum distance: 0.04 nm
          Maximum distance: 1.0 nm
          Reference distance: 1.0 nm
          Log Gaussian begin: -3.218876
          Log Gaussian end: 0.006724119
          Interval for log Gaussian: 0.0512
          Sigma for log gaussian: 0.3
          Number of basis functions: 64
          Rescale the range of RBF to (-1,1).
       Calculate distance: Yes
       Calculate bond: No
       Feature dimension: 128
-----------------------------------------------------------------------

In [6]:
tot_params = 0
for i, param in enumerate(net.get_parameters()):
    tot_params += param.size
    print(i, param.name, param.shape)
print('Total parameters: ', tot_params)

0 model.atom_embedding.embedding_table (64, 128)
1 model.dis_filter.linear.weight (128, 64)
2 model.dis_filter.linear.bias (128,)
3 model.dis_filter.residual.nonlinear.mlp.0.weight (128, 128)
4 model.dis_filter.residual.nonlinear.mlp.0.bias (128,)
5 model.dis_filter.residual.nonlinear.mlp.1.weight (128, 128)
6 model.dis_filter.residual.nonlinear.mlp.1.bias (128,)
7 model.interactions.0.positional_embedding.norm.gamma (128,)
8 model.interactions.0.positional_embedding.norm.beta (128,)
9 model.interactions.0.positional_embedding.x2q.weight (128, 128)
10 model.interactions.0.positional_embedding.x2k.weight (128, 128)
11 model.interactions.0.positional_embedding.x2v.weight (128, 128)
12 model.interactions.0.multi_head_attention.output.weight (128, 128)
13 model.interactions.0.pondering.dense.weight (1, 384)
14 model.interactions.0.pondering.dense.bias (1,)
15 model.interactions.1.positional_embedding.norm.gamma (128,)
16 model.interactions.1.positional_embedding.norm.beta (128,)
17 model.i

In [7]:
n_epoch = 8
repeat_time = 1
batch_size = 32

ds_train = ds.NumpySlicesDataset(
    {'R': train_data['R'], 'Z': train_data['Z'], 'E': train_data['E'][:, idx]}, shuffle=True)
ds_train = ds_train.batch(batch_size, drop_remainder=True)
ds_train = ds_train.repeat(repeat_time)
loss_network = WithLabelLossCell('RZE', net, nn.MAELoss())

WithLabelLossCell with input type: RZE


In [8]:
lr = 1e-3
optim = nn.Adam(params=net.trainable_params(), learning_rate=lr)

In [9]:
model = Model(loss_network, optimizer=optim)

In [10]:
monitor_cb = LossMonitor(16)

In [11]:
outdir = 'Tutorial_C02'
params_name = outdir + '_' + net.model_name
config_ck = CheckpointConfig(
    save_checkpoint_steps=32, keep_checkpoint_max=64, append_info=[net.hyper_param])
ckpoint_cb = ModelCheckpoint(
    prefix=params_name, directory=outdir, config=config_ck)

In [12]:
print("Start training ...")
beg_time = time.time()
model.train(n_epoch, ds_train, callbacks=[monitor_cb, ckpoint_cb], dataset_sink_mode=False)
end_time = time.time()
used_time = end_time - beg_time
m, s = divmod(used_time, 60)
h, m = divmod(m, 60)
print("Training Fininshed!")
print("Training Time: %02d:%02d:%02d" % (h, m, s))

Start training ...
epoch: 1 step: 16, loss is 154.4375
epoch: 1 step: 32, loss is 114.806640625
epoch: 2 step: 16, loss is 84.23046875
epoch: 2 step: 32, loss is 71.810546875
epoch: 3 step: 16, loss is 143.591796875
epoch: 3 step: 32, loss is 81.3046875
epoch: 4 step: 16, loss is 104.359375
epoch: 4 step: 32, loss is 56.529296875
epoch: 5 step: 16, loss is 57.82421875
epoch: 5 step: 32, loss is 85.54296875
epoch: 6 step: 16, loss is 40.58984375
epoch: 6 step: 32, loss is 37.1875
epoch: 7 step: 16, loss is 52.048828125
epoch: 7 step: 32, loss is 48.0390625
epoch: 8 step: 16, loss is 40.001953125
epoch: 8 step: 32, loss is 56.359375
Training Fininshed!
Training Time: 00:01:04
